In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
%matplotlib inline
from matplotlib.colorbar import colorbar_factory
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV


In [2]:
df1=pd.read_excel(r"D:\Новая папка\\Users\\HP\Загрузки\hw_data_composite/X_nup.xlsx")

In [3]:
df2=pd.read_excel(r"D:\Новая папка\\Users\\HP\Загрузки\hw_data_composite/X_bp.xlsx")

In [4]:
df=df2.merge(df1, how="inner")
df1.dropna(inplace=True)

In [5]:
df.drop('Unnamed: 0',axis = 1, inplace = True)
print(df.shape)
#df.head()

(1023, 13)


In [6]:
Q1 = df2.quantile(q=.25)
Q3 = df2.quantile(q=.75)
IQR = Q3 - Q1
df = df2[~((df2 < (Q1-1.5*IQR)) | (df2 > (Q3+1.5*IQR))).any(axis=1)]

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
columns = df.columns
df = scaler.fit_transform(np.array(df))
df = pd.DataFrame(df, columns=columns)
#df.head()

In [9]:
# scaler_X = MinMaxScaler()
# scaler_y = MinMaxScaler() 
# y = scaler_y.fit_transform(df[y_label])
# X = scaler_X.fit_transform(df.drop(y_label, axis = 1))
y_label = ['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа']
y = df[y_label]
X = df.drop(y_label, axis = 1)
y.shape,X.shape

((961, 2), (961, 9))

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,shuffle=True)

In [11]:
loss_df = pd.DataFrame(columns=['target','model','MSE','R2'])

In [14]:
from sklearn.metrics import mean_squared_error

In [21]:
from sklearn.linear_model import LinearRegression

In [27]:
from sklearn.metrics import r2_score

In [31]:
# модуль упругости при растяжении и прочность
df =pd.DataFrame([])
linear_model = LinearRegression()
linear_model.fit(X_train,  y_train)
y_pred = linear_model.predict(X_test)


model_name = 'Linear Regression'
def add_loss(loss_df, model_name):
  MSE = mean_squared_error(y_test, y_pred)
  R2 = r2_score(y_test, y_pred)
    
  
  df = pd.DataFrame({'model':model_name,\
                                'target':['Модуль упругости и Прочность'],\
                                'MSE':MSE,\
                                'R2':R2})
  # print(df.head())
  loss_df = pd.concat([loss_df, df],ignore_index=True)
  return loss_df
  loss_df = add_loss(loss_df, model_name)
# loss_df 'Linear Regression'

In [25]:
print(MSE)

0.03832604774083058


In [35]:
print(R2)

0.002139893472607235


In [63]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
grid = { 
    'estimator__C':np.arange(1.0,5.5,0.5),
    'estimator__epsilon':np.arange(0.1,1.1,0.1)}
linear_model = LinearRegression()
linear_model.fit(X_train,  y_train)
y_pred = linear_model.predict(X_test)

model = MultiOutputRegressor(SVR())
gs = GridSearchCV(model, grid, n_jobs=-1, cv=10)
gs.fit(X_train,  y_train)
model = gs.best_estimator_

MSE = mean_squared_error(y_test,y_pred)
R2 = r2_score(y_test,y_pred)
model_name = 'SVR'
loss_df = add_loss(loss_df,model_name)
def predicted_plot(y_test, y_pred,model_name):
     MSE = mean_squared_error(y_test, y_pred)
predicted_plot(y_test, y_pred,model_name)

In [71]:
predictions = gs.predict(X_test)
 

In [67]:
mae = np.mean(abs(predictions - y_test))

C:\Users\HP\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [68]:
#средняя абсолютная ошибка test выборки
print(mae)

Модуль упругости при растяжении, ГПа    0.160363
Прочность при растяжении, МПа           0.155501
dtype: float64


In [70]:
predictionss = gs.predict(X_train)


In [72]:
mae_train=np.mean(abs(predictionss - y_train))

C:\Users\HP\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [73]:
print(mae_train)

Модуль упругости при растяжении, ГПа    0.154989
Прочность при растяжении, МПа           0.147873
dtype: float64


In [36]:
print(); print(gs.best_estimator_.get_params())


{'estimator__C': 1.0, 'estimator__cache_size': 200, 'estimator__coef0': 0.0, 'estimator__degree': 3, 'estimator__epsilon': 0.5, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf', 'estimator__max_iter': -1, 'estimator__shrinking': True, 'estimator__tol': 0.001, 'estimator__verbose': False, 'estimator': SVR(epsilon=0.5), 'n_jobs': None}


In [37]:
loss_df.head()

,target,model,MSE,R2
0,Модуль упругости и Прочность,SVR,0.038326,0.00214


In [40]:
gs.best_estimator_

MultiOutputRegressor(estimator=SVR(epsilon=0.5))

In [58]:
loss_df

,target,model,MSE,R2
0,Модуль упругости и Прочность,SVR,0.038326,0.00214
1,Модуль упругости и Прочность,SVR,0.038326,0.00214
2,Модуль упругости и Прочность,SVR,0.038326,0.00214
